# Analyse des tâches
Le but de ce chapitre est d'approfondir [l'analyse des tâches](https://docs.computecanada.ca/wiki/Running_jobs/fr#Suivi_des_t.C3.A2ches) en attente, en cours et les tâches terminées.

Remplissons la liste de tâches gérées par l'ordonnanceur :

```Bash
cat scripts/sleep-300s.sh
sbatch scripts/sleep-300s.sh  # Répéter au besoin
```

## Tâches en attente
Pour afficher les tâches en attente, on utilise la commande :
* [`squeue -t pending`](https://slurm.schedmd.com/squeue.html) pour toutes les tâches en attente
* `squeue -t pending -u $USER` pour seulement **vos** tâches en attente

La commande suivante affiche les tâches en attente triées par priorité descendante :
```Bash
squeue -t pending --sort=-p --format="%.16i %.9p %.8u %.16a %.18P %.10l %.4C %.7m"
```
* À propos des différentes colonnes :
  * `%i` : identifiant numérique de la tâche
  * `%p` : priorité de la tâche (entre 0.0 et 1.0)
  * `%u` : nom d'utilisateur
  * `%a` : compte de calcul utilisé
  * `%P` : [partition de la grappe](https://docs.computecanada.ca/wiki/Job_scheduling_policies/fr#Pourcentages_des_n.C5.93uds_disponibles)
  * `%l` : temps demandé
  * `%C` : nombre de processeurs demandé
  * `%m` : mémoire demandée

Dans tous les cas, l'utilisation de `squeue` crée une certaine charge
sur l'ordonnanceur (à cause du grand nombre de tâches à traiter),
alors il vaut mieux **se limiter à un appel par minute** sur les grappes nationales.
* Si jamais il vous venait l'idée d'utiliser la commande `watch`, **ne le faites pas!**

### Partition et autres spécifications d'une tâche
Une chose à savoir : vos tâches sont généralement en compétition avec des [tâches de la même partition](https://docs.computecanada.ca/wiki/Job_scheduling_policies/fr#Pourcentages_des_n.C5.93uds_disponibles).
Ainsi, **une tâche CPU n'a aucun impact sur le temps d'attente d'une tâche GPU**.
Vous pouvez utiliser la commande suivante pour obtenir le détail de votre tâche, incluant la partition à laquelle elle est assignée :
```Bash
scontrol show job -dd <jobid>
```

### Politique d'ordonnancement
Ce qu'il faut aussi comprendre de la [politique d'ordonnancement](https://docs.computecanada.ca/wiki/Job_scheduling_policies/fr) :
* **La priorité de votre tâche dépend majoritairement de l'utilisation récente** de votre groupe de recherche
* Pour le reste, la priorité dépend aussi du temps d'attente, mais dans une très faible proportion

L'information concernant **l'utilisation récente du groupe** est donnée par la commande [`sshare`](https://docs.computecanada.ca/wiki/Job_scheduling_policies/fr#Priorisation_selon_la_juste_part) :
```Bash
sshare -l -A def-prof1_cpu -u prof1,grad2,postdoc3
sshare -l  # Tous les comptes
```
* À propos des différentes colonnes :
  * Plus le `LevelFS` du groupe est élevé, plus la priorité est élevée
  * L'utilisation est considérée normale si `LevelFS` est autour de 1.0
  * Si `LevelFS` est proche de 0.0, la priorité du groupe sera basse
* L'utilisation récente perd de son importance avec le temps qui passe.
  * Son **importance est progressivement réduite** avec un ratio réduisant l'impact de 50% à chaque 7 jours
  * Par conséquent, le `LevelFS` et la priorité augmentent à nouveau lorsque le groupe reste en attente

## Tâches actives
Pour afficher les tâches actives, on utilise la commande :
* [`squeue -t running`](https://slurm.schedmd.com/squeue.html) pour toutes les tâches actives
* `squeue -t running -u $USER` pour seulement **vos** tâches actives

La commande suivante affiche les tâches actives triées selon le temps qui leur reste :
```Bash
squeue -t running --sort=+M --format="%.14i %.18P %.10M %.4C %.7m %R"
```
* À propos des différentes colonnes :
  * `%i` : identifiant numérique de la tâche
  * `%P` : partition de la grappe
  * `%M` : temps de calcul écoulé
  * `%C` : nombre de processeurs demandé
  * `%m` : mémoire demandée
  * `%R` : liste des noeuds alloués

### Valider le comportement d'une tâche active
Pendant que vos tâches sont actives, vous pouvez vous connecter par SSH aux noeuds de calcul correspondants afin de valider que l'exécution se passe bien :
```Bash
sbatch scripts/inv-mat.sh
```

* Identifier le ou les noeud(s) avec : `squeue -u $USER`
* Connexion avec : `ssh <nom_noeud>`
* Inspection avec `top`, `htop`, `nvtop` et/ou `nvidia-smi` :
  * Est-ce que vos processus s'exécutent avec un **pourcentage de 100%?**
  * Est-ce que vos processus parallèles s'exécutent avec un **pourcentage de $n$ * 100%**, où $n$ est le nombre de processeurs par tâche Slurm?
  * Est-ce que les **accélérateurs** sont utilisés à leur plein potentiel?
    * Note : il est préférable de regarder la **consommation en Watts** plutôt que le pourcentage affiché
  * Est-ce que le **noeud de calcul** semble pleinement utilisé (tous les processeurs, la mémoire et/ou les accélérateurs)?

Toutes ces informations devraient vous permettre d'ajuster vos scripts de soumission et d'optimiser l'exécution parallèle de vos différents calculs.

## Tâches terminées
Lorsqu'une [tâche est complétée](https://docs.computecanada.ca/wiki/Running_jobs/fr#T.C3.A2ches_termin.C3.A9es), la commande la plus simple pour obtenir des informations sur cette tâche est la commande :
```Bash
seff <jobid>
```
Autrement, c'est aussi possible d'utiliser la commande Slurm
[`sacct`](https://slurm.schedmd.com/sacct.html) :
```Bash
sacct -j <jobid> --format=JobID,JobName,MaxRSS,Elapsed
```
Encore une fois, ces informations permettent d'ajuster vos scripts et de mieux déterminer vos besoins en ressources.

Ultimement, l'utilisation totale du groupe est disponible sur CCDB :
* https://ccdb.computecanada.ca/me/group_usage

En tenant compte de l'utilisation passée, il devient possible d'estimer ses besoins pour le [Concours d'allocation de ressources](https://www.computecanada.ca/page-daccueil-du-portail-de-recherche/acces-aux-ressources/concours-dallocation-des-ressources/?lang=fr).

## Points à retenir
* Lister nos **tâches en attente** : `squeue -t pending -u $USER`
  * Le détail d'une tâche : `scontrol show job -dd <jobid>`
  * L'utilisation récente : `sshare -l -A def-prof1_cpu -u $USER`
* Lister nos **tâches actives** : `squeue -t running -u $USER`
  * Possibilité d'aller inspecter les tâches directement sur les noeuds de calcul
* Lister nos **tâches terminées** : `sacct`
  * Pour une tâche précise : `seff <jobid>`
  * Ou bien : `sacct -j <jobid> --format=JobID,JobName,MaxRSS,Elapsed`